In [36]:
import networkx as nx
import numpy as np
from typing import List, Set, Tuple, Union
from scipy.sparse import lil_matrix
from sklearn.cluster import AgglomerativeClustering
import math 
from copy import deepcopy

Please note that this is a very raw version of the algorithm: the code is now organized as a number of functions that should be organized more coherently in an actual class or integrated in an existing library. It works, though.


<H2> UTILS </H2
>

In [37]:
def create_graph_from_nodes_list(nodes: Set, edge_list: np.ndarray, create_using = nx.DiGraph) -> nx.Graph:
    if create_using == nx.Graph:
        G = nx.Graph()
    elif create_using == nx.DiGraph:
        G = nx.DiGraph()
    G.add_nodes_from(nodes)
    G.add_weighted_edges_from(edge_list)
    return G

#Gs is the list of graphs (layers) of our multiplex network.
#layer_labels is the list of labels associated to each layer. so layer_labels[0] will be the label of the edges in Gs[0].
def create_multiplex_from_graphs(Gs: List[nx.Graph], layer_labels: List[str], is_directed: bool = True, has_attr: bool = True) -> nx.MultiGraph:
    #assuming we are dealing with multiplex networks -> for every G in Gs, <V> is the same
    multiplex_nodes = Gs[0].nodes.items()
    if is_directed:
        mp = nx.MultiDiGraph()
    else:
        mp = nx.MultiGraph()
    mp.add_nodes_from(multiplex_nodes)
    
    for i, G in enumerate(Gs):
        mp_edges = get_edges_for_multiplex_graph(G, layer_labels[i], has_attr)
        mp.add_edges_from(mp_edges)
    
    layer_mapping = {i: label for i, label in enumerate(layer_labels)}
    mp.graph['layers'] = layer_mapping
    return mp

def get_edges_for_multiplex_graph(G: nx.Graph, key: None, has_attr: bool = True) -> List[Tuple]:
    edges = nx.to_edgelist(G)
    mp_edges = []
    for edge in edges:
        if has_attr and key:
            mp_edges.append(tuple_insert(edge, 2, key))
        elif key and has_attr == False:
            mp_edges.append(edge + (key,))
        else:
            mp_edges.append(edge)
    return mp_edges

def tuple_insert(tup,pos,ele):
    tup = tup[:pos]+(ele,)+tup[pos:]
    return tup

In [3]:
# ============================================================================
# EXAMPLE AND TESTING
# ============================================================================

def create_tagarelli_network(return_as_list: bool = True) -> Union[nx.Graph, List[List]]:

    nodelist = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

    clustering = [
        [['1', '2', '3'], ['4', '5', '6'], ['7', '8', '9', '10']],
        [['1', '2', '3'], ['4', '6', '7', '9']],
        [['2', '4', '5', '6'], ['7', '8', '9']]
    ]

    G = nx.MultiGraph()


    # Define layers
    layers = { 0: 'L1',
              1: 'L2',
              2: 'L3'
    }
    G.graph['layers'] = layers

    if return_as_list: 
        G1 = nx.Graph()
        G2 = nx.Graph()
        G3 = nx.Graph()

        G1.add_nodes_from(nodelist)
        G2.add_nodes_from(nodelist)
        G3.add_nodes_from(nodelist)

        #LAYER 1
        G1.add_edge('1', '2', weight = 1.0)
        G1.add_edge('2', '3', weight = 1.0)
        G1.add_edge('2', '6', weight = 1.0)
        G1.add_edge('4', '5', weight = 1.0)
        G1.add_edge('4', '6', weight = 1.0)
        G1.add_edge('5', '6', weight = 1.0)
        G1.add_edge('5', '7', weight = 1.0)
        G1.add_edge('6', '9', weight = 1.0)
        G1.add_edge('7', '8', weight = 1.0)
        G1.add_edge('7', '9', weight = 1.0)
        G1.add_edge('8', '9', weight = 1.0)
        G1.add_edge('8', '10', weight = 1.0)
        G1.add_edge('9', '10', weight = 1.0)

        #LAYER 2
        G2.add_edge('1', '2', weight = 1.0)
        G2.add_edge('1', '3', weight = 1.0)
        G2.add_edge('2', '3', weight = 1.0)
        G2.add_edge('3', '4', weight = 1.0)
        G2.add_edge('4', '6', weight = 1.0)
        G2.add_edge('4', '9', weight = 1.0)
        G2.add_edge('6', '7', weight = 1.0)
        G2.add_edge('6', '9', weight = 1.0)
        G2.add_edge('7', '9', weight = 1.0)

        #LAYER 3
        G3.add_edge('2', '4', weight = 1.0)
        G3.add_edge('2', '6', weight = 1.0)
        G3.add_edge('4', '5', weight = 1.0)
        G3.add_edge('4', '6', weight = 1.0)
        G3.add_edge('5', '6', weight = 1.0)
        G3.add_edge('5', '7', weight = 1.0)
        G3.add_edge('7', '8', weight = 1.0)
        G3.add_edge('7', '9', weight = 1.0)
        G3.add_edge('8', '9', weight = 1.0)

        return [G1, G2, G3], clustering
    else:
        raise ValueError('Not implemented yet, return as list')


<H2> M-EMCD IMPLEMENTATION </H2>

In [ ]:
# ================================================================ #
# COMMUNITIES MANIPULATION
# ================================================================ #

def get_nodes_in_communities(communities: List[int], G: nx.Graph) -> List[str]:
    nodes = np.array(list(G.nodes()))
    #print(nodes)
    n_clusters = np.unique(communities)
    communities_by_nodes = []
    for i in n_clusters:
        mask = np.array([True if com == i else False for com in communities], dtype = bool)
        
        nodes_subset = nodes[mask]
        #print(nodes_subset)
        communities_by_nodes.append(nodes_subset)
    return communities_by_nodes

def get_intercomm_edges_as_list(com: dict) -> List:
    com_edges_list = []
    for cj, intercomm_edges_ci_cj in com.items():
        for layer in intercomm_edges_ci_cj.values():
            if len(layer) > 0:
                com_edges_list.extend(layer)
    return com_edges_list

def create_consensus_structure(G: nx.MultiGraph, intracomm_edges: dict, intercomm_edges: dict) -> nx.MultiGraph:
    if G.is_directed():
        consensus_structure = nx.MultiDiGraph()
    else:
        consensus_structure = nx.MultiGraph()
    communities = []
    consensus_nodes = list(G.nodes(data = False))
    consensus_structure.add_nodes_from(consensus_nodes)
    #add intracommunity edges and create internal community structure
    for community in intracomm_edges.values():
        comm_edges = []
        for layer in community:
            if len(community[layer]) > 0:
                consensus_structure.add_edges_from(community[layer])
                comm_edges.extend([(e[0], e[1], e[2]) for e in community[layer]])
        communities.append(consensus_structure.edge_subgraph(comm_edges))
    #add intercommunity edges to consensus structure
    for community in intercomm_edges.keys():
        intercomm_edges_list = get_intercomm_edges_as_list(intercomm_edges[community])
        consensus_structure.add_edges_from(intercomm_edges_list)

    return consensus_structure, communities    
        

# ================================================================ #
# EDGE MANIPULATION FUNCTIONS
# ================================================================ #

def get_edges_between_u_v(G: nx.MultiDiGraph, u: str, v: str, return_data: bool = False) -> List:
    edges = G.edges(u, data = return_data, keys = True)
    uv_edges = []
    for edge in edges:
        if edge[1] == v:
            uv_edges.append(edge)
    return uv_edges

def get_edges_between_u_v_by_layer(G: nx.MultiGraph, u: str, v: str, layers: List[str] = None, return_data: bool = False) -> dict:
    edges = G.edges(u, data = return_data, keys = True)
    by_layer = {k: [] for k in G.graph['layers'].values()}
    for edge in edges:
        if edge[1] == v:
            if layers:
                if edge[2] in layers:
                    by_layer[edge[2]].append(edge)
            else:
                by_layer[edge[2]].append(edge)
    return by_layer

def get_edges_in_layer_for_node(node, G: nx.MultiDiGraph, layer: str, return_data: bool = True) -> List:
    edges_for_node_in_layer = [edge for edge in G.edges(node, keys = True, data = return_data) if edge[2] == layer]
    return edges_for_node_in_layer

def get_nodes_induced_intercommunity_edges(G: nx.MultiDiGraph, com_1: nx.MultiDiGraph, com_2: nx.MultiDiGraph, layer: str) -> List:
    edges_between = []
    for u in com_1.nodes(data = False):
        for v in com_2.nodes(data = False):
            edges_uv = get_edges_between_u_v(G, u, v, True)
            if edges_uv:
                edges_between.extend([edge for edge in edges_uv if edge[2] == layer])
    return edges_between

def get_intracommunity_edges_ccemcd(G: nx.MultiGraph, mij: dict, community: nx.MultiGraph):
    community_nodes = list(community.nodes(data = False))
    intracomm_edges = {layer: [] for layer in G.graph['layers'].values()}
    #intracomm_edges = []
    for node in community_nodes:
        if mij[node]:
            for k in mij[node].keys():
                #print(f'checking edges for {node} - {k}')
                uv_edges = get_edges_between_u_v_by_layer(G, node, k, None, True)
                for layer in G.graph['layers'].values():
                    intracomm_edges[layer].extend(uv_edges[layer])
                #print(uv_edges)
    return intracomm_edges

def get_intracommunity_edges_matrix_ccemcd(G: nx.MultiGraph, communities: List[nx.MultiGraph], mij: dict) -> dict:
    intracomm_edges_dict = {i: {layer: [] for layer in G.graph['layers'].values()} for i in range(len(communities))}
    for i, com in enumerate(communities):
        intracomm_edges = get_intracommunity_edges_ccemcd(G, mij, com)
        if len(intracomm_edges) > 0:
            for layer in G.graph['layers'].values():
                intracomm_edges_dict[i][layer].extend(intracomm_edges[layer])
            #intracomm_edges_dict[i].extend(get_intracommunity_edges_ccemcd(G, mij, com))
    return intracomm_edges_dict

def get_intercommunity_edges_ccemcd(G: nx.MultiGraph, inverse_mij: dict, communities: List[nx.MultiGraph]) -> dict:
    intercomm_edges = {c:
                        {k:
                          {layer: [] for layer in G.graph['layers'].values()} 
                          for k, _ in enumerate(communities)}
                        for c, _ in enumerate(communities)}
    if len(communities) <= 1:
        return intercomm_edges
    #intercomm_edges = {c: {} for c, _ in enumerate(communities)}
    for key_i, value in inverse_mij.items():
        c1_idx  = get_community_index_for_node(key_i, communities)
        # print(f'i : {key_i}')
        for key_j in value.keys():
            # print(f'      j : {key_j}')
            c2_idx = get_community_index_for_node(key_j, communities)
            #print(c2_idx)
            to_select = [layer for layer, intercoms in value[key_j].items() if len(intercoms) > 0]
            c1_c2_edges = get_edges_between_u_v_by_layer(G, key_i, key_j, to_select, True)
            # print(f'             edges between i, j: {c1_c2_edges}')
            # print(intercomm_edges)
            for layer in to_select:
                intercomm_edges[c1_idx][c2_idx][layer].extend(c1_c2_edges[layer])                               
    return intercomm_edges

def get_community_index_for_node(node: str, communities: List[nx.MultiGraph]) -> int:
    for i, com in enumerate(communities):
        community_nodes = list(com.nodes(data = False))
        if node in community_nodes:
            return i




def add_edges_to_community(com: nx.MultiDiGraph, edges) -> nx.MultiDiGraph:
    c_updated = deepcopy(com)
    c_updated.add_edges_from(edges)
    return c_updated

def compute_edge_difference_set(e1, e2):
    return [e for e in e1 if e not in e2]

# ================================================================ #
# CO-ASSOCIATION MATRIX CONSTRUCTION 
# ================================================================ #

def compute_mij(node_i: str, node_j: str, G: nx.MultiDiGraph, clustering: List[List], n_layers: int, linkage_constraint: bool = False) -> float:
    #print(node_i)
    mij = {layer: [] for layer in G.graph['layers'].values()}
    inverse_mij = {layer: [] for layer in G.graph['layers'].values()}
    #inverse_mij = []
    #mij = []
    for i in range(n_layers):
        current_layer = G.graph['layers'][i]
        for j, com in enumerate(clustering[i]):
            #if both are not in the current community we cannot say anything, 
            #maybe they share the next one. But if only one of node_i or node_j are in the community,
            #then surely they won't share the community (they are disjoint), yet they may potentially be linked in this layer
            share_comm = (node_i in com) and (node_j in com)
            one_of_two_in_comm =  node_i in com and (not node_j in com) or node_j in com and (not node_i in com)
            if (not share_comm) and (one_of_two_in_comm):
                if linkage_constraint:
                    if G.has_edge(node_i, node_j, key = current_layer):
                        #inverse_mij.append(i)
                        inverse_mij[current_layer].append(j)
                #inverse_mij is not defined in case we do not have the linkage constraint since mij contains all the edges connecting two nodes
            elif share_comm:
                if linkage_constraint:
                    if G.has_edge(node_i, node_j, key = current_layer):
                        #mij.append((i, j))
                        mij[current_layer].append(j)
                else:
                    #mij.append((i,j))
                    mij[current_layer].append(j)
        
    return mij, inverse_mij


#THE VERSION TO USE, OPTIMIZED FOR TIME AND SPACE 
def compute_coass_sparse_matrix(G: nx.Graph, clustering: List[List[str]], as_distance_matrix: bool = True, linkage_constraint: bool = False, theta: float =  None):
    nodes = list(G.nodes())
    n_nodes = len(nodes)
    n_layers = len(clustering)
    #use sparse matrices for efficiency
    coass_matrix = lil_matrix((n_nodes, n_nodes), dtype= np.float32)
    #mij contains, for every pair of nodes, a reference to the layers in which i,j share the same community and are also connected by an edge
    mij = {str(k): {} for k in nodes}
    #inverse mij contains, for every pair of nodes, a reference to the layers in which i,j do not share the same community but are connected by an edge
    inverse_mij = {str(k): {} for k in nodes}
    #flags for updates
    added_mij = False
    added_inverse_mij = False
    #start the loop
    for i, node_i in enumerate(nodes):
        for j, node_j in enumerate(nodes):
            #reset flags
            added_mij = False
            added_inverse_mij = False
            if node_i != node_j:
                #compute mij and inverse_mij to keep track of intracommunity and intercommunity edges in the clustering steps
                mij_tmp, inverse_mij_tmp = compute_mij(node_i, node_j, G, clustering, n_layers, linkage_constraint)
                len_mij = sum([len(mij_tmp[layer]) for layer in list(mij_tmp.keys())])
                len_inverse_mij  = sum([len(inverse_mij_tmp[layer]) for layer in list(mij_tmp.keys())])
                #if len(mij_tmp) > 0:
                if len_mij > 0:
                    #add only if we have mij, for space reasong
                    #consider the linkage constraint threshold theta for cc-emcd
                    if theta:
                        #if(len(mij_tmp)/n_layers) >= theta:
                        if len_mij/n_layers > theta:
                            mij[node_i][node_j] = mij_tmp
                            added_mij = True
                        else:
                            #if we have a threshold and we do not surpass it, empty mij so that coass[i, j] = 0 (or 1 if as distance)
                            #mij_tmp = []
                            len_mij = 0
                    else:
                        mij[node_i][node_j] = mij_tmp
                        added_mij = True
                    if as_distance_matrix:
                        #coass_matrix[i, j] =  1 - (len(mij_tmp) / n_layers)
                        coass_matrix[i, j] =  1 - (len_mij / n_layers)
                    else:
                        #coass_matrix[i, j] =  len(mij_tmp) / n_layers
                        coass_matrix[i, j] =  1 - (len_mij / n_layers)
                else:
                #if mij is 0 then the distance is 1, if we do not want the distance matrix we can skip
                    if as_distance_matrix:
                        coass_matrix[i, j] = 1
                #add if we have links when we do not share communities and update flag
                #if len(inverse_mij_tmp) > 0:
                if len_inverse_mij > 0:
                    inverse_mij[node_i][node_j] = inverse_mij_tmp
                    added_inverse_mij = True
               #symmetry in case the graph is undirected
                if not G.is_directed():
                    if added_mij:
                        mij[node_j][node_i] = mij[node_i][node_j]
                        coass_matrix[j, i] = coass_matrix[i, j]
                    if added_inverse_mij:
                        inverse_mij[node_i][node_j] = {k: list(set(v)) for k, v in inverse_mij[node_i][node_j].items()}
    return coass_matrix, mij, inverse_mij

# ================================================================ #
# DEGREE COMPUTATION FUNCTIONS
# ================================================================ #

#compute the internal degree of a community 
#assuming the community only has internal edges
def get_community_internal_layer_degree(com: nx.MultiDiGraph, layer: str) -> float:
    #layer_edges = get_edges_in_layer(com, layer)
    layer_internal_degree = 0.0
    for node in com.nodes():
        neighbors = com.neighbors(node)
        for neighbor in neighbors:
            if neighbor in com.nodes() and com.has_edge(node, neighbor, layer):
                layer_internal_degree += com[node][neighbor][layer]['weight']
    return layer_internal_degree

#compute the layer degree of a graph
def get_layer_degree(G: nx.MultiDiGraph, layer: str) -> float:
    layer_edges = get_edges_in_layer(G, layer, False)
    print(layer_edges)
    layer_subgraph = G.edge_subgraph(layer_edges)
    return sum([layer_subgraph.degree(node, weight = 'weight') for node in layer_subgraph])

#given a graph com and a layer, returns the edges of com in the layer specified by the layer param.
# get_data specifies whether the edges data also have to be returned or not
def get_edges_in_layer(com: nx.MultiDiGraph, layer: str, return_data: bool = True) -> list:
    edges_in_layer = [edge for edge in com.edges(keys = True, data = return_data) if edge[2] == layer]
    return edges_in_layer

def get_node_layer_degree(node, G: nx.MultiGraph, layer: str, weight: str = 'weight') -> float:
    edges_for_node_in_layer = get_edges_in_layer_for_node(node, G, layer, True)
    degree = np.sum([e[3][weight] for e in edges_for_node_in_layer])
    return degree

def D_sq_l_c_new(d_lc_dict: dict) -> float:
    coms_degrees = []
    for com in d_lc_dict.keys():
        com_degree = 0
        for l_deg in d_lc_dict[com].values():
            com_degree += math.pow(l_deg, 2)
        coms_degrees.append(com_degree)
    return np.sum(coms_degrees)

#compute a dict such that dict[i][l] = degree for community i at layer l
def compute_D_l_c_dict(consensus: nx.MultiDiGraph, communities: List[nx.MultiDiGraph], layers: list[str], weight: str = 'weight') -> dict:
    degree_dict = {i: {l: 0 for l in layers} for i, _ in enumerate(communities)}
    for i, com in enumerate(communities):
        degree_dict[i] = compute_community_D_l_c(consensus, com, layers, weight)
    return degree_dict

def compute_community_D_l_c(consensus, community, layers: str, weight: str = 'weight') -> float:
    layers_degrees_dict = {l: 0 for l in layers}
    com_nodes = list(community.nodes(data = False))
    for layer in layers:
        layer_degree = 0
        for node in com_nodes:
            node_degree_in_layer = get_node_layer_degree(node, consensus, layer, weight)
            layer_degree += node_degree_in_layer
        layers_degrees_dict[layer] = layer_degree
    return layers_degrees_dict

def D_VL_new(G:nx.MultiGraph) -> float:
    nodes_degree = dict(G.degree())
    whole_degree = np.sum([deg for deg in nodes_degree.values()])
    return whole_degree

#we are keeping track of both the consensus structure (graph made of communities with internal and external edges) 
#and separated communities(communities with only the internal edges)
def D_int_lc_new(coms: List[nx.MultiGraph]) -> float:
    coms_degrees = []
    #compute internal degree for each community
    for com in coms:
        com_degree = dict(com.degree())
        coms_degrees.append(np.sum([deg for deg in com_degree.values()]))
    return np.sum(coms_degrees)



# ================================================================ #
# MODULARITY UPDATE FUNCTIONS
# ================================================================ #

def modularity(d_vl: float, d_int_lc: float, d_sq_l_c: float) -> float:
    if d_vl == 0:
        return 0.0
    return (1/d_vl * d_int_lc) - (1/math.pow(d_vl, 2) * d_sq_l_c)

def modularity_within_update(d_vl: float, d_int_lc: float, d_sq_l_c: float, d_li_Cj: float, new_edges: List, is_weighted: bool = True) -> float:
    """
    Modularity update for adding edges within communities (Equation 5).
    """
    if is_weighted:
        n_new_edges = sum([edge[3]['weight'] for edge in new_edges])
    else:
        n_new_edges = len(new_edges)

    denominator = d_vl + 2*n_new_edges
    if denominator == 0:
        return 0.0
    return ((d_int_lc + 2*n_new_edges) / (denominator)) - ((d_sq_l_c + 4 * n_new_edges * (n_new_edges + d_li_Cj))/math.pow(denominator, 2))

def modularity_between_update(d_vl: float, d_int_lc: float, d_sq_l_c: float, d_li_Cj: float,d_li_Ch: float,
                               new_edges_between: List, is_weighted: bool = True, weight: str = 'weight' ) -> float:
    """
    Modularity update for adding edges between communities (Equation 7).
    """
    if is_weighted:
        #n_new_edges_between = [sum([edge[3]['weight'] for edge in edges]) for edges in new_edges_between]
        n_new_edges_between = [edge[3][weight] for edge in new_edges_between]
    else:
        n_new_edges_between = [1 for edge in new_edges_between]

    ne = sum(n_new_edges_between)
    denominator = d_vl + 2*ne
    #sq_sum_of_nek = sum([math.pow(nek, 2) for nek in n_new_edges_between])
    sq_ne = math.pow(ne, 2)
    weighted_degree = ne * (d_li_Cj + d_li_Ch)
    
    first_term = d_int_lc / denominator
    second_term = (d_sq_l_c + 2*sq_ne + 2*weighted_degree)/math.pow(denominator, 2)
    return ('a', first_term - second_term)

def modularity_between_update_remove(d_vl: float, d_int_lc: float, d_sq_l_c: float,
                                    d_li_Cj: float, d_li_Ch: float,
                                    edges_to_remove: List,
                                    is_weighted: bool = True, weight: str = 'weight') -> float:
    """
    Modularity update for removing edges between communities (Equation 9 of the paper).
    """
    # if len(edges_to_remove) == 0:
    #     return ('r', d_int_lc / d_vl - d_sq_l_c / (d_vl ** 2))
    if is_weighted:
        #n_edges_to_remove = [sum([edge[3]['weight'] for edge in edges]) for edges in edges_to_remove]
        n_edges_to_remove = [edge[3][weight] for edge in edges_to_remove]
    else:
        n_edges_to_remove = [1 for edge in edges_to_remove]

    ne = sum(n_edges_to_remove)
    denominator = d_vl - 2*ne
    #sq_sum_of_nek = sum([math.pow(nek, 2) for nek in n_edges_to_remove])
    sq_ne = math.pow(ne, 2)
    weighted_degree = 2 * ne * (d_li_Cj + d_li_Ch)
    #weighted_degree_of_neighbors = 2* sum([nek*d_li_ck for nek, d_li_ck in zip(n_edges_to_remove, d_li_Cs)])
    first_term = d_int_lc / denominator
    #second_term = (d_sq_l_c + math.pow(ne, 2) + sq_sum_of_nek - 2*ne*d_li_Cj - weighted_degree_of_neighbors)/math.pow(denominator, 2)
    second_term = (d_sq_l_c + 2*sq_ne - weighted_degree)/ math.pow(denominator, 2)

    return ('r', first_term - second_term)

def modularity_between_update_both(d_vl: float, d_int_lc: float,
                                   d_li_Cj: float, d_li_Ch: float, j: int,
                                   cj: nx.MultiDiGraph, consensus: nx.MultiDiGraph, d_lc_dict: dict,
                                   layers: List[str], current_layer: str,
                                   edges_to_remove: List,
                                   edges_to_add: List,
                                   is_weighted: bool = False, 
                                   weight: str = 'weight') -> float:
    
    """
    Modularity update for both removing and adding edges between communities (chained).
    
    This chains two operations:
    1. First remove current inter-community edges
    2. Then add new inter-community edges
    
    Args:
        d_vl: Total degree of multilayer graph
        d_int_lc: Cumulative internal degree
        d_sq_l_c: Cumulative squared degree
        d_li_Cj: Degree of community Cj in layer Li
        d_li_Ch: Degree of community Ch in layer Li
        edges_to_remove: Edges to remove
        edges_to_add: Edges to add
        G: Multilayer graph
        is_weighted: Whether edges have weights
    
    Returns:
        Updated modularity after both operations
    """
    consensus_copy = consensus.copy()
    d_lc_dict_copy = d_lc_dict.copy()
    consensus.add_edges_from(edges_to_add)
    cj_dict_l_c = compute_community_D_l_c(consensus_copy, cj, layers, weight)
    d_lc_dict_copy[j] = cj_dict_l_c
    d_sq_l_c_plus = D_sq_l_c_new(d_lc_dict_copy)
    modularity_plus_minus = modularity_between_update_remove(d_vl, d_int_lc, d_sq_l_c_plus, cj_dict_l_c[current_layer], d_li_Ch, edges_to_remove,
                                                             is_weighted, weight)
    return ('b', modularity_plus_minus[1])
    
# ================================================================ #
# UPDATE FUNCTIONS
# ================================================================ #

def update_community(G: nx.MultiDiGraph, current_Q: float, cj: nx.MultiDiGraph, layer: str, d_vl: float, d_int_lc: float, d_sq_l_c: float, d_li_Cj: float) ->Union[nx.MultiDiGraph, float, List]:
    #compute edges to add
    cj_nodes_subgraph = G.subgraph(cj.nodes(data = False))
    #available_edges = get_edges_in_layer(cj_nodes_subgraph, layer)
    new_edges = compute_edge_difference_set(get_edges_in_layer(cj_nodes_subgraph, layer, True), get_edges_in_layer(cj, layer, True))
    # print(f'available edges: {get_edges_in_layer(cj_nodes_subgraph, layer, True)}')
    # print(f'edges already in community: {get_edges_in_layer(cj, layer, True)}')
    # print(f'new edges to be added: {new_edges}')
    #if there are no new edges we can return the current cj and current modularity
    if len(new_edges) == 0:
        return cj, current_Q, []
    #else we add the edges to the community and compute the update
    #cj_prime = deepcopy(cj)

    cj_prime = cj.copy()
    #cj_prime.add_edges_from(new_edges)
    modularity_update = modularity_within_update(d_vl, d_int_lc, d_sq_l_c, d_li_Cj, new_edges, is_weighted = True)
    return cj_prime, modularity_update, new_edges

def update_community_structure(G: nx.MultiDiGraph, consensus: nx.MultiDiGraph, cj: nx.MultiDiGraph, ch:nx.MultiDiGraph,
                                h: int, j: int, cj_neighbors: dict, layers: List[str], layer: str,
                                d_vl: float, d_int_lc: float, d_sq_l_c: float, d_li_Cj: float, d_li_Ch: float, d_lc_dict: dict,
                                is_weighted: bool = True, weight: str = 'weight'):
    #get the neighbors between cj and ch at layers l
    consensus_edges_between_cj_ch_for_l = cj_neighbors[h][layer]
    #get the edges that are in the original graph but not in our consensus structure
    edges_in_graph_between_cj_ch = get_nodes_induced_intercommunity_edges(G, cj, ch, layer)
    e_prime = compute_edge_difference_set(edges_in_graph_between_cj_ch, consensus_edges_between_cj_ch_for_l)
    #compute the removal update in modularity
    removal_modularity_update = modularity_between_update_remove(d_vl, d_int_lc, d_sq_l_c, d_li_Cj, d_li_Ch,
                                                                  consensus_edges_between_cj_ch_for_l,  is_weighted, weight)
    #compute the addition update in modularity
    addition_modularity_update = modularity_between_update(d_vl, d_int_lc, d_sq_l_c, d_li_Cj, d_li_Ch,
                                                           e_prime, is_weighted, weight)
    #compute both operations jointly: first add the new edges and compute the increase in modularity and then remove the old
    # ones and compute the removal update on the new updated community
    both_modularity_update = modularity_between_update_both(d_vl, d_int_lc, d_li_Cj, d_li_Ch, j, cj, consensus, d_lc_dict,
                                                            layers, layer, consensus_edges_between_cj_ch_for_l, e_prime, is_weighted, weight)
    best_update = max([removal_modularity_update, addition_modularity_update, both_modularity_update], key = lambda x: x[1])
    consensus_copy = consensus.copy()
    edges_between_cj_ch_l_copy = cj_neighbors[h][layer].copy()
    if best_update[0] == 'r' and len(consensus_edges_between_cj_ch_for_l) > 0:
        edges_between_cj_ch_l_copy = []
        for edge in consensus_edges_between_cj_ch_for_l:
            consensus_copy.remove_edge(edge[0], edge[1], edge[2])
    elif best_update[0] == 'a':
        edges_between_cj_ch_l_copy.extend(e_prime)
        consensus_copy.add__edges_from(e_prime)
    elif best_update[0] == 'b' and len(consensus_edges_between_cj_ch_for_l) > 0:
        edges_between_cj_ch_l_copy = e_prime
        for edge in consensus_edges_between_cj_ch_for_l:
            consensus_copy.remove_edge(edge[0], edge[1], edge[2])
        consensus_copy.add_weighted_edges_from(e_prime)
    
    return (consensus_copy, edges_between_cj_ch_l_copy, h, best_update[1])

# ================================================================ #
# C-EMCD ALGORITHM
# ================================================================ #

#compute single_linkage on coass_matrix -> cut on threshold -> compute nodes subgraphs (communities)
def c_emcd(coass_matrix: np.ndarray, theta: float, G: nx.MultiDiGraph) -> List[nx.Graph]:
    theta_dist = 1 - theta
    consensus = []
    agg = AgglomerativeClustering(n_clusters = None, metric = 'precomputed', linkage = 'single', distance_threshold = theta_dist)
    communities = agg.fit_predict(coass_matrix)
    communities_by_nodes = get_nodes_in_communities(communities, G)
    for subset in communities_by_nodes:
        consensus.append(G.subgraph(subset))
    return consensus

def cc_emcd_fast(G: nx.MultiGraph, base_cluster: List[nx.MultiGraph], theta: float) -> Union[List[nx.MultiGraph], dict, dict]:
    theta_dist = 1 - theta
    print('=====================================================================')
    print('Computing co-association matrix, mij, inverse mij...')
    coass_matrix, mij, inverse_mij = compute_coass_sparse_matrix(G, base_cluster, True, True, theta)
    print('Done!')
    print('=====================================================================')
    print(f'Computing Consensus with respect to distance matrix, theta: {theta_dist}')
    coms = c_emcd(coass_matrix.toarray(), theta_dist, G)
    print('Done!')
    print('=====================================================================')
    print(f'Computing the intracommunity edges matrix...')
    intracommunity_edges = get_intracommunity_edges_matrix_ccemcd(G, coms, mij)
    print('Done!')
    print('=====================================================================')
    print('Computing the intercommunity edges matrix...')
    intercommunity_edges = get_intercommunity_edges_ccemcd(G, inverse_mij, coms)
    print('Done!')
    return coms, intracommunity_edges, intercommunity_edges

# ================================================================ #
# M-EMCD ALGORITHM
# ================================================================ #
def m_emcd(G: nx.MultiDiGraph, coms: List[str], theta: float, max_iteration: int, threshold: float, weight: str = 'weight') -> List[nx.MultiDiGraph]:
    #STARTUP PHASE
    #compute coassociation matrix and lowerbound for the consensus using the cc-emcd
    # print("Computing co-association matrix...")
    # mij, coass_matrix = compute_coass_matrix(G, coms)
    print('Computing lower-bound consensus through CC-EMCD...')
    cc_emcd_coms, intracomm_edges, intercomm_edges = cc_emcd_fast(G, coms, theta)
    consensus_lower, consensus_comms = create_consensus_structure(G, intracomm_edges, intercomm_edges)
    # print('initial consensus structure')
    # for edge in consensus_lower.edges(keys = True):
    #     print(edge)
    consensus_star = consensus_lower
    print(f'Consensus communities: {len(consensus_comms)}')
    iteration = 0
    delta_Q = np.inf
    layers = G.graph['layers'].values()
    #Compute initial degrees and modularity
    print('Computing initial degrees and modularity...')
    d_vl = D_VL_new(G)
    #d_int_lc = D_int_lc(consensus_lower, layers)
    d_int_lc = D_int_lc_new(consensus_comms)
    #d_sq_l_c = D_sq_l_c(consensus_lower, layers)
    d_lc_dict = compute_D_l_c_dict(consensus_star, consensus_comms, layers, weight)
    d_sq_l_c = D_sq_l_c_new(d_lc_dict)
    prev_Q = modularity(d_vl, d_int_lc, d_sq_l_c)
    prev_prev_Q = prev_Q
    print(f"Initial modularity: {prev_Q:.4f}")
    while iteration < max_iteration:
        print(f"\n--- Iteration {iteration} ---")
        iteration += 1
        improved = False
        #start the optimization loop
        for value in layers:
            print(f'Computing updates for layer {value}')
            #compute the degrees and modularity
            intracommunity_updates = []
            comm_candidates = []
            added_edges = []
            #loop over every community in the current consensus
            for j, cj in enumerate(consensus_comms):
                print(f'Computing updates for community c{j}')
                #print(cj)
                d_li_cj = d_lc_dict[j][value]
                com_candidate, Q_intracomm_update, new_edges = update_community(G, prev_Q, cj, value, d_vl, d_int_lc, d_sq_l_c, d_li_cj)
                #cj_candidate = update_community(G, prev_Q, cj, value, d_vl, d_int_lc, d_sq_l_c, d_li_cj)
                comm_candidates.append(com_candidate)
                intracommunity_updates.append(Q_intracomm_update)
                added_edges.append(new_edges)
            #Select the best update
            print(f'  Intra-community updates for layer {value}: {intracommunity_updates}')
            j_star = np.argmax(intracommunity_updates)
            best_intracomm_Q = intracommunity_updates[j_star]
            if best_intracomm_Q > prev_Q:
                improvement = best_intracomm_Q - prev_Q
                print(f"  Intra-community update: Community {j_star}, "
                      f"ΔQ = +{improvement:.6f}")
                #update consensus
                c_j_star = comm_candidates[j_star]
                #consensus_star[j_star] = c_j_star
                consensus_comms[j_star] = c_j_star
                #consensus_star.add_edges_from([(e[0], e[1], e[2], e[3]) for e in added_edges[j_star]])
                consensus_star.add_edges_from(added_edges[j_star])
                #update modularity
                prev_Q = best_intracomm_Q
                improved = True
                #startup of the second update  
                #degree of the improved community 
                d_lc_dict[j_star] = compute_community_D_l_c(consensus_star, c_j_star, layers, weight)  
                d_int_lc = D_int_lc_new(consensus_comms)  
                d_sq_l_c = D_sq_l_c_new(d_lc_dict)         
                d_li_cj_star = d_lc_dict[j_star][value]
                #We updated C_j and thus can proceed in updating the community structure
                print(f'Computing community structure updates for community at index {j_star}, at layer {value}')
                cj_neighbors = intercomm_edges[j_star]
                intercomm_candidates = []
                best_intercomm_Q = -np.inf
                for h in cj_neighbors.keys():
                    if h == j_star:
                        continue
                    if len(cj_neighbors[h][value]) > 0:
                        ch = consensus_comms[h]
                        d_li_ch = d_lc_dict[h][value]
                        ch_candidate = update_community_structure(G, consensus_star, c_j_star, ch, h, j_star, cj_neighbors, layers, value,
                                                                        d_vl, d_int_lc, d_sq_l_c, d_li_cj_star, d_li_ch, d_lc_dict, True, weight)
                        #save also the original community index so the intercommunity edges matrix can be updated
                        intercomm_candidates.append(ch_candidate)
                #select the best update
                if len(intercomm_candidates) > 0:
                    best_intercomm_update = max(intercomm_candidates, key = lambda x: x[3])
                    best_intercomm_Q = best_intercomm_update[3]
                #if we are incrementing wrt the previous update (cj*)
                if best_intercomm_Q > prev_Q:
                    #print('entro mai qua?')
                    #update the intercomm_edges matrix
                    h_star = best_intercomm_update[2]
                    edges_update = best_intercomm_update[1]
                    print(edges_update)
                    print(len(best_intercomm_update[0].edges()))
                    cj_neighbors[h_star][value] = edges_update
                    #update consensus structure
                    consensus_star = best_intercomm_update[0]
                    #update d_li_cj_star
                    d_lc_dict[j_star] = compute_community_D_l_c(consensus_star, c_j_star, layers, weight) 
                    improvement = best_intercomm_Q - prev_Q
                    prev_prev_Q = prev_Q
                    prev_Q = best_intercomm_Q
                    #current_Q = 
                    improved = True
                    print(f"Inter-community update for layer {value}: ΔQ = +{improvement:.6f}")

        # Recompute statistics after community update
        # d_int_lc = D_int_lc(consensus_star, layers)
        # d_sq_l_c = D_sq_l_c(consensus_star, layers)
        d_int_lc = D_int_lc_new(consensus_comms)
        d_sq_l_c = D_sq_l_c_new(d_lc_dict)
        #current_Q = modularity(d_vl, d_int_lc, d_sq_l_c)
        delta_Q = abs(prev_Q - prev_prev_Q)
    
        print(f"End of iteration {iteration}: Q = {prev_Q:.4f}, ΔQ = {delta_Q:.6f}")
        
        if not improved or delta_Q < threshold:
            print(f"\nConverged after {iteration} iterations")
            break
        #prev_Q = current_Q

    #final_Q = modularity(d_vl, d_int_lc, d_sq_l_c)
    print(f"\nFinal modularity: {prev_Q:.4f}")
    #print(f"Number of communities: {len()}")
    
    return consensus_star, consensus_comms, prev_Q

<H2> USAGE EXAMPLE </H2>

The create_tagarelli_network() method provides a small example taken from the paper Ensemble-based Community Detection In Multilayer Networks by Tagarelli et al., in which the algorithm has been first presented (https://doi.org/10.1007/s10618-017-0528-8), in particular the network proposed in figure 4.

In [44]:
tagarelli_graph, tagarelli_clustering = create_tagarelli_network()

The create_multiplex_from_graphs converts a list of single layer graphs to a networkx multiGraph. We also need to pass the name of the layers so that these can be encoded in the graph structure.

In [45]:
mg_tagarelli = create_multiplex_from_graphs(tagarelli_graph, ['L1', 'L2', 'L3'], False, True)

The M-EMCD algorithm can be executed by simply callind the m_emcd function: the needed parameters are a multilayer graph, the initial clustering (obtained via single layer community detection), the theta parameter, the max number of iterations and convergence threshold, and a string indicating the name of the attribute that contains the weight of the edges in the graph.


In [46]:
tagarelli_memcd_consensus ,tagarelli_memcd_coms, tagarelli_memcd_modularity = m_emcd(mg_tagarelli, tagarelli_clustering, 0.6, 100, 0.0000001, 'weight')

Computing lower-bound consensus through CC-EMCD...
Computing co-association matrix, mij, inverse mij...
Done!
Computing Consensus with respect to distance matrix, theta: 0.4
Done!
Computing the intracommunity edges matrix...
Done!
Computing the intercommunity edges matrix...
Done!
Consensus communities: 4
Computing initial degrees and modularity...
Initial modularity: 0.5010

--- Iteration 0 ---
Computing updates for layer L1
Computing updates for community c0
Computing updates for community c1
Computing updates for community c2
Computing updates for community c3
  Intra-community updates for layer L1: [np.float64(0.5010405827263267), np.float64(0.5010405827263267), np.float64(0.5010405827263267), np.float64(0.5010405827263267)]
Computing updates for layer L2
Computing updates for community c0
Computing updates for community c1
Computing updates for community c2
Computing updates for community c3
  Intra-community updates for layer L2: [np.float64(0.5010405827263267), np.float64(0.5131